In [6]:
import pymongo
import pandas as pd
import numpy as np
from pandas_datareader import data as web
from datetime import datetime as dt

In [7]:
import pymongo

In [8]:
import yaml
with open(".passwords/mongo_pwd.yml", 'r') as stream:
    try:
        auth = yaml.load(stream)
    except yaml.YAMLError as exc:
        print(exc)

In [9]:
client = pymongo.MongoClient('mongodb://%s:%s@127.0.0.1' % ('guest', auth['password']))
db = client["1-gram_time"]

In [10]:
df = pd.DataFrame(list(db["tweets"].find({ "word": "and" })))
df.head(5)

,_id,counts,rank,time,word
0,5c328d4a5920610636fb2ca0,2374312,8,2018-05-20,and
1,5c328d4d5920610636fcb3dc,2011354,9,2014-07-06,and
2,5c328d505920610636fe3b09,130232,9,2010-02-22,and
3,5c328d545920610636ffe431,1827301,9,2018-05-03,and
4,5c328d575920610636016df4,1917554,9,2015-02-17,and


In [11]:
def load(word):
    df = pd.DataFrame(list(db["tweets"].find({ "word": word})))
    #df['date'] = [dt.strptime(time, '%Y-%m-%d') for time in df['time']]
    #df['date'] = dt.strftime(df['time'], '%dth %b %Y, %I:%M%p')
    #df['rank'] = df['rank'].astype(int)
    df.sort_values(by="time", inplace=True)
    return df

In [12]:
from plotly import __version__
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly.graph_objs as go
init_notebook_mode(connected=True)

In [13]:
def buildtimeseries(wordlist):
    
    traces = []
    
    for word in wordlist:
        traces.append(go.Scatter(
                x=load(word)['time'],
                text=load(word)['time'],
                y=load(word)['rank'],
                name = str(word),
                opacity = 0.8))

    data = traces

    layout = dict(
        title = str(wordlist),
        yaxis=dict(
            type='log',
            autorange='reversed',
            fixedrange=True
            )
        )

    fig = dict(data=data, layout=layout)
    config={'showLink': False, 'scrollZoom': True}
    iplot(fig, config=config)

In [14]:
buildtimeseries(["#revengeporn","#4chan"])

In [15]:
buildtimeseries(["اغفر","حسابك","الذين","وهي","عاجل","تفضل","معرفة","التويتر","مميزة","زار"])